In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
import math

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#Packet based features
df_1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets_iot/Benign_packetbase/BenignTraffic.csv', on_bad_lines='skip', engine='python') #skip bad lines
df_2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets_iot/Benign_packetbase/BenignTraffic1.csv', on_bad_lines='skip', engine='python') #skip bad lines
df_3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets_iot/Benign_packetbase/BenignTraffic2.csv', on_bad_lines='skip', engine='python') #skip bad lines
df_4 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets_iot/Benign_packetbase/BenignTraffic3.csv', on_bad_lines='skip', engine='python') #skip bad lines

In [4]:
data=pd.concat([df_1,df_2,df_3,df_4],axis=0)

In [5]:
#Data shape
data.shape

(526976, 135)

In [16]:
#Data info
print(data.info())
#Duplication data
print(data.duplicated())

<class 'pandas.core.frame.DataFrame'>
Index: 526976 entries, 0 to 126145
Columns: 135 entries, stream to l3_ip_dst_count
dtypes: float64(118), int64(1), object(16)
memory usage: 546.8+ MB
None
0         False
1         False
2         False
3         False
4         False
          ...  
126141    False
126142    False
126143    False
126144    False
126145    False
Length: 526976, dtype: bool


In [77]:
#Data Description
data.head(20)

,stream,src_mac,dst_mac,src_ip,dst_ip,src_port,dst_port,inter_arrival_time,time_since_previously_displayed_frame,port_class_dst,...,sum_p,min_p,max_p,med_p,average_p,var_p,q3_p,q1_p,iqr_p,l3_ip_dst_count
0,0,Arlo Q Indoor Camera,3c:18:a0:41:c3:a0,192.168.137.175,99.81.244.93,56891.0,443.0,0.000000,0.000000,1.0,...,58160.0,2908.0,2908.0,2908.0,2908.00,0.000000e+00,2908.0,2908.0,0.0,1.0
1,0,Arlo Q Indoor Camera,3c:18:a0:41:c3:a0,192.168.137.175,99.81.244.93,56891.0,443.0,0.000164,0.000164,1.0,...,58160.0,2908.0,2908.0,2908.0,2908.00,0.000000e+00,2908.0,2908.0,0.0,1.0
2,0,Arlo Q Indoor Camera,3c:18:a0:41:c3:a0,192.168.137.175,99.81.244.93,56891.0,443.0,0.000269,0.000105,1.0,...,56712.0,1460.0,2908.0,2908.0,2835.60,1.048352e+05,2908.0,2908.0,0.0,1.0
3,0,Arlo Q Indoor Camera,3c:18:a0:41:c3:a0,192.168.137.175,99.81.244.93,56891.0,443.0,0.000414,0.000145,1.0,...,55264.0,1460.0,2908.0,2908.0,2763.20,1.986351e+05,2908.0,2908.0,0.0,1.0
4,0,Arlo Q Indoor Camera,3c:18:a0:41:c3:a0,192.168.137.175,99.81.244.93,56891.0,443.0,0.001800,0.001386,1.0,...,53816.0,1460.0,2908.0,2908.0,2690.80,2.813997e+05,2908.0,2908.0,0.0,1.0
5,0,Arlo Q Indoor Camera,3c:18:a0:41:c3:a0,192.168.137.175,99.81.244.93,56891.0,443.0,0.001961,0.000161,1.0,...,55264.0,1460.0,4356.0,2908.0,2763.20,4.193408e+05,2908.0,2908.0,0.0,1.0
6,0,Arlo Q Indoor Camera,3c:18:a0:41:c3:a0,192.168.137.175,99.81.244.93,56891.0,443.0,0.002129,0.000168,1.0,...,53816.0,1460.0,4356.0,2908.0,2690.80,5.021054e+05,2908.0,2908.0,0.0,1.0
7,0,Arlo Q Indoor Camera,3c:18:a0:41:c3:a0,192.168.137.175,99.81.244.93,56891.0,443.0,0.008073,0.005944,1.0,...,51405.0,497.0,4356.0,2908.0,2570.25,7.376284e+05,2908.0,2546.0,362.0,1.0
8,0,Arlo Q Indoor Camera,3c:18:a0:41:c3:a0,192.168.137.175,99.81.244.93,56891.0,443.0,0.008075,0.000002,1.0,...,49957.0,497.0,4356.0,2908.0,2497.85,7.909834e+05,2908.0,1460.0,1448.0,1.0
9,0,Arlo Q Indoor Camera,3c:18:a0:41:c3:a0,192.168.137.175,99.81.244.93,56891.0,443.0,0.008173,0.000098,1.0,...,49957.0,497.0,4356.0,2908.0,2497.85,7.909834e+05,2908.0,1460.0,1448.0,1.0


In [6]:
#columns of the dataset
data.columns

Index(['stream', 'src_mac', 'dst_mac', 'src_ip', 'dst_ip', 'src_port',
       'dst_port', 'inter_arrival_time',
       'time_since_previously_displayed_frame', 'port_class_dst',
       ...
       'sum_p', 'min_p', 'max_p', 'med_p', 'average_p', 'var_p', 'q3_p',
       'q1_p', 'iqr_p', 'l3_ip_dst_count'],
      dtype='object', length=135)

In [7]:
#Cheking of nan values
print(data.isna().sum())
#Percentage of nan_values
pourcentage_nan=(data.isna().sum()/data.shape[0])*100
print(pourcentage_nan)
#Features which has nan
print(pourcentage_nan[pourcentage_nan>0])

stream                 0
src_mac                0
dst_mac                0
src_ip             23670
dst_ip             23670
                   ...  
var_p                  4
q3_p                   4
q1_p                   4
iqr_p                  4
l3_ip_dst_count        4
Length: 135, dtype: int64
stream             0.000000
src_mac            0.000000
dst_mac            0.000000
src_ip             4.491666
dst_ip             4.491666
                     ...   
var_p              0.000759
q3_p               0.000759
q1_p               0.000759
iqr_p              0.000759
l3_ip_dst_count    0.000759
Length: 135, dtype: float64
src_ip                4.491666
dst_ip                4.491666
src_port              0.000190
dst_port              0.000190
inter_arrival_time    0.000190
                        ...   
var_p                 0.000759
q3_p                  0.000759
q1_p                  0.000759
iqr_p                 0.000759
l3_ip_dst_count       0.000759
Length: 132, dtype: fl

In [5]:
#Drop na in serc_ip and dst_ip
data.dropna(subset=['src_ip', 'dst_ip'], inplace=True)

In [6]:
#For tensor construction reasons, we will remove all none frequents stream, src_mac,dst_mac
def remove_NF(series, th=1):
    freq = series.value_counts()
    return freq[freq >= th].index.tolist()

L1 = remove_NF(data['stream'], th=1000)
L2 = remove_NF(data['src_mac'], th=1000)
L3 = remove_NF(data['dst_mac'], th=1000)
L4=remove_NF(data['src_ip'], th=50)
L5=remove_NF(data['dst_ip'], th=50)

In [7]:
#Filtered data
data_filtered = data[
    data['stream'].isin(L1) &
    data['src_mac'].isin(L2) &
    data['dst_mac'].isin(L3)&
    data['src_ip'].isin(L4)&
    data['dst_ip'].isin(L5)
]

In [8]:
#construction of The tensor: We will construct several tensors
#Each of them will capture the normal
subdata_1=data_filtered[['stream','src_ip','dst_ip','inter_arrival_time']]
subdata_2=data_filtered[['stream','src_ip','dst_ip','src_port','inter_arrival_time']]
subdata_3=data_filtered[['stream','src_ip','src_port','dst_port','dst_ip','inter_arrival_time']]
subdata_4=data_filtered[['stream','src_mac','dst_mac','src_ip','dst_ip','src_port','dst_port','inter_arrival_time','sum_p']]

In [10]:
#Tensor construction
#For subdata_1 : We construct it in two ways : Dense and Sparse
#Dense tensor
def create_tensor(df):
    df_copy = df.copy()

    # 1. Création des mappings pour chaque colonne
    mappings = {}
    shape = []

    for col in df_copy.columns:
        uniques = sorted(df_copy[col].unique())
        mappings[col] = {val: idx for idx, val in enumerate(uniques)}
        shape.append(len(uniques))

    # 2. Initialisation du tenseur
    tensor = np.zeros(shape, dtype=np.int8)

    # 3. Remplissage du tenseur
    for _, row in df_copy.iterrows():
        try:
            idx = tuple(mappings[col][row[col]] for col in df_copy.columns)
            tensor[idx] = 1
        except KeyError:
            continue  # ignore si une valeur ne figure pas dans le mapping

    return tensor, mappings

tensor, mappings = create_tensor(subdata_1)

# Afficher la forme du tenseur
print(f"Forme du tenseur : {tensor.shape}")
nonzero_count = np.count_nonzero(tensor)
print('number of nonezero',nonzero_count )

Forme du tenseur : (103, 181, 183, 355515)
number of nonezero 355551


In [ ]:
#In a Sparse way


In [11]:
#10 percent of subdata_1
subdata_1_10=subdata_1.sample(frac=0.1, random_state=42)

In [12]:
#Tensor construction
#For subdata_1 : We construct it in two ways : Dense and Sparse
#Dense tensor
def create_tensor(df):
    df_copy = df.copy()

    # 1. Création des mappings pour chaque colonne
    mappings_10 = {}
    shape_10 = []

    for col in df_copy.columns:
        uniques = sorted(df_copy[col].unique())
        mappings_10[col] = {val: idx for idx, val in enumerate(uniques)}
        shape_10.append(len(uniques))

    # 2. Initialisation du tenseur
    tensor_10 = np.zeros(shape_10, dtype=np.int8)

    # 3. Remplissage du tenseur
    for _, row in df_copy.iterrows():
        try:
            idx = tuple(mappings_10[col][row[col]] for col in df_copy.columns)
            tensor_10[idx] = 1
        except KeyError:
            continue  # ignore si une valeur ne figure pas dans le mapping

    return tensor_10, mappings_10

tensor_10, mappings_10 = create_tensor(subdata_1_10)

# Afficher la forme du tenseur
print(f"Forme du tenseur : {tensor_10.shape}")
nonzero_count_10 = np.count_nonzero(tensor_10)
print('number of nonezero',nonzero_count_10 )

Forme du tenseur : (103, 176, 172, 35577)
number of nonezero 35577


In [96]:
#Groupe the inter_arrival_time in bins according to similar_values
# Bin values by rounding to the nearest 0.00001
epsilon = 0.00001
subdata_1_10['binned_inter_arrival'] = (subdata_1_10['inter_arrival_time'] ).round(5)

In [14]:
!git config --global user.email 'mitsoumayonelpricejospin@gmail.com'
!git config --global user.name 'mitsouma'

In [ ]:
!git clone https://github.com/mitsouma/IOT_Anomalies_Detection_System_Based_on_Tensor_Decomposition.git
